In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

In [2]:
df = pd.read_csv('summary.mts10k.mbs50.buckets.none.mutrans.data.single.5000.1.50.None.S_50.0.nueff_100.tau_100.0.T_50000_10000.s0.csv', index_col=0)

In [3]:
def export_pip_table(dataframe, num_rows=20, filename='allele_level.pdf', by='PIP'):
    dataframe = dataframe.sort_values(by=[by], ascending=False)
    dataframe = dataframe.iloc[:num_rows]
    
    beta = ['{:.3f} ± {:.3f}'.format(b, 1.96 * s) for b, s in zip(dataframe['Beta'].values,
                                                                   dataframe['BetaStd'].values)]
    dataframe.loc[dataframe.index, 'Beta'] = pd.Series(beta, index=dataframe.index)
    dataframe.loc[dataframe.index, 'PIP'] = np.round(dataframe['PIP'].values, 4)
    dataframe = dataframe.reset_index(level=0)
    dataframe.index = np.arange(1, 1 + dataframe.values.shape[0])    
    dataframe = dataframe[['index', 'PIP', 'Beta']]
    dataframe.columns = ['Mutation', 'PIP', 'Beta']
    inrbd = lambda x: int(x) >= 319 and int(x) <= 541
    rbd = [m.split(':')[0] == 'S' and inrbd(m.split(':')[1][1:-1]) for m in dataframe['Mutation'].values]  #(319, 541)
    dataframe['RBD'] = ['Yes' if m == True else '' for m in rbd]
    dataframe = dataframe[['Mutation', 'RBD', 'PIP', 'Beta']]
    dfi.export(dataframe, filename)
    print(dataframe)
    return dataframe

In [4]:
export_pip_table(df, num_rows=25, filename='pip_ranked_alleles.png', by='PIP')